In [7]:
# ===============================================
# KLASIFIKASI CHD DENGAN SMOTE-IPF DAN SVM
# ===============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning essentials
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score, confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE

# Random seed untuk reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Libraries loaded - Ready for CHD Classification with SMOTE-IPF")

✅ Libraries loaded - Ready for CHD Classification with SMOTE-IPF


In [8]:
# ===============================================
# 1. DATA LOADING & BASIC ANALYSIS
# ===============================================

# Load dataset
df = pd.read_csv("framingham.csv")
print(f"📊 Dataset loaded: {df.shape[0]} samples, {df.shape[1]} features")

# Quick data quality check
missing_count = df.isnull().sum().sum()
print(f"🔍 Missing values: {missing_count}")

# Check class distribution (imbalance analysis)
if "TenYearCHD" in df.columns:
    class_dist = df['TenYearCHD'].value_counts()
    imbalance_ratio = class_dist[0] / class_dist[1]
    
    print(f"\n⚖️  CLASS DISTRIBUTION:")
    print(f"   No CHD (0): {class_dist[0]} ({class_dist[0]/len(df)*100:.1f}%)")
    print(f"   CHD (1): {class_dist[1]} ({class_dist[1]/len(df)*100:.1f}%)")
    print(f"   Imbalance Ratio: {imbalance_ratio:.1f}:1")
    print(f"   Status: {'HIGHLY IMBALANCED' if imbalance_ratio > 5 else 'IMBALANCED'}")

print("\n✅ Data analysis completed - Imbalanced dataset confirmed")

📊 Dataset loaded: 4240 samples, 16 features
🔍 Missing values: 645

⚖️  CLASS DISTRIBUTION:
   No CHD (0): 3596 (84.8%)
   CHD (1): 644 (15.2%)
   Imbalance Ratio: 5.6:1
   Status: HIGHLY IMBALANCED

✅ Data analysis completed - Imbalanced dataset confirmed


In [9]:
# ===============================================
# 2. DATA PREPROCESSING
# ===============================================

# Handle missing values with median imputation
imputer = SimpleImputer(strategy="median")
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

print(f"🔧 Data preprocessing completed")
print(f"   Missing values after imputation: {df.isnull().sum().sum()}")
print("✅ Dataset ready for feature selection")

🔧 Data preprocessing completed
   Missing values after imputation: 0
✅ Dataset ready for feature selection


In [10]:

feature_columns = [col for col in df.columns if col != 'TenYearCHD']
correlations = df.corr()['TenYearCHD'].drop('TenYearCHD').abs().sort_values(ascending=False)

# Select top 4 features
selected_features = list(correlations.head(4).index)

print("🎯 SELECTED FEATURES (Top 4 by correlation):")
for i, (feature, corr) in enumerate(correlations.head(4).items(), 1):
    print(f"   {i}. {feature:15s}: {corr:.3f}")

# Prepare data with selected features
X = df[selected_features]
y = df['TenYearCHD']

print(f"\n📊 FEATURE REDUCTION: {len(feature_columns)} → {len(selected_features)} features")
print("✅ Feature selection completed")

🎯 SELECTED FEATURES (Top 4 by correlation):
   1. age            : 0.225
   2. sysBP          : 0.216
   3. prevalentHyp   : 0.177
   4. diaBP          : 0.145

📊 FEATURE REDUCTION: 15 → 4 features
✅ Feature selection completed


In [11]:


# Split data (80:20 ratio)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"📊 DATA SPLIT:")
print(f"   Training set: {X_train_scaled.shape}")
print(f"   Test set: {X_test_scaled.shape}")
print(f"   Train distribution: {np.bincount(y_train)}")
print(f"   Test distribution: {np.bincount(y_test)}")
print("✅ Data splitting & scaling completed")

📊 DATA SPLIT:
   Training set: (3392, 4)
   Test set: (848, 4)
   Train distribution: [2877  515]
   Test distribution: [719 129]
✅ Data splitting & scaling completed


In [ ]:
# Cell C: SVM tanpa class_weight (mungkin mirip dengan baseline dosen)
from sklearn.svm import SVC

svm_plain = SVC(kernel='linear', C=1.0, probability=True, random_state=42)  # gamma dihapus karena linear
svm_plain.fit(X_train_scaled, y_train)
y_pred_plain = svm_plain.predict(X_test_scaled)

print("SVM (no class_weight) Classification Report:\n", classification_report(y_test, y_pred_plain, digits=3))
print("Accuracy:", accuracy_score(y_test, y_pred_plain))
print("F1-score (pos=1):", f1_score(y_test, y_pred_plain, pos_label=1))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_plain))



SVM (no class_weight) Classification Report:
               precision    recall  f1-score   support

         0.0      0.848     1.000     0.918       719
         1.0      0.000     0.000     0.000       129

    accuracy                          0.848       848
   macro avg      0.424     0.500     0.459       848
weighted avg      0.719     0.848     0.778       848

Accuracy: 0.847877358490566
F1-score (pos=1): 0.0
Confusion Matrix:
 [[719   0]
 [129   0]]


In [13]:
# SMOTE + SVM
import numpy as np
from sklearn.metrics import classification_report, f1_score

# Terapkan SMOTE pada data training
smote = SMOTE(sampling_strategy=0.5, random_state=42, k_neighbors=3)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

print("Jumlah setelah SMOTE:", X_resampled.shape, np.bincount(y_resampled))

# SVM dengan parameter yang lebih optimal
svm_smote = SVC(kernel='linear', C=0.5, class_weight='balanced', random_state=42)  # Lebih simpel dan cepat
svm_smote.fit(X_resampled, y_resampled)
y_pred_smote = svm_smote.predict(X_test_scaled)

print("\n=== HASIL SVM + SMOTE ===")
print("Classification Report:\n", classification_report(y_test, y_pred_smote, digits=3))
print("F1-Score:", f1_score(y_test, y_pred_smote))

# Bandingkan dengan baseline SVM
print("\n=== PERBANDINGAN ===")
print("F1-Score Baseline SVM:", f1_score(y_test, y_pred_plain))
print("F1-Score SVM + SMOTE:", f1_score(y_test, y_pred_smote))
print("Improvement:", f1_score(y_test, y_pred_smote) - f1_score(y_test, y_pred_plain))

Jumlah setelah SMOTE: (4315, 4) [2877 1438]

=== HASIL SVM + SMOTE ===
Classification Report:
               precision    recall  f1-score   support

         0.0      0.894     0.645     0.750       719
         1.0      0.225     0.574     0.323       129

    accuracy                          0.634       848
   macro avg      0.559     0.609     0.536       848
weighted avg      0.792     0.634     0.685       848

F1-Score: 0.3231441048034934

=== PERBANDINGAN ===
F1-Score Baseline SVM: 0.0
F1-Score SVM + SMOTE: 0.3231441048034934
Improvement: 0.3231441048034934

=== HASIL SVM + SMOTE ===
Classification Report:
               precision    recall  f1-score   support

         0.0      0.894     0.645     0.750       719
         1.0      0.225     0.574     0.323       129

    accuracy                          0.634       848
   macro avg      0.559     0.609     0.536       848
weighted avg      0.792     0.634     0.685       848

F1-Score: 0.3231441048034934

=== PERBANDINGAN ==

In [14]:
# ===============================================
# 5. SMOTE-IPF IMPLEMENTATION (PROPOSED METHOD)
# ===============================================

class SMOTE_IPF(BaseEstimator):
    """
    SMOTE-IPF: SMOTE with Iterative Partitioning Filter
    
    Combines SMOTE oversampling with IPF noise filtering to improve
    synthetic sample quality for imbalanced CHD classification.
    """
    
    def __init__(self, sampling_strategy=1.0, smote_k=5, ipf_k=3, 
                 max_iter=10, random_state=None, verbose=False):
        self.sampling_strategy = sampling_strategy
        self.smote_k = smote_k
        self.ipf_k = ipf_k
        self.max_iter = max_iter
        self.random_state = random_state
        self.verbose = verbose
    
    def fit_resample(self, X, y):
        """Apply SMOTE + IPF filtering"""
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y).ravel()
        
        # Step 1: SMOTE Oversampling
        smote = SMOTE(
            sampling_strategy=self.sampling_strategy,
            k_neighbors=self.smote_k,
            random_state=self.random_state
        )
        X_smote, y_smote = smote.fit_resample(X, y)
        
        if self.verbose:
            print(f"🔄 After SMOTE: {X_smote.shape[0]} samples, {np.bincount(y_smote.astype(int))}")
        
        # Step 2: IPF Noise Filtering
        n_original = X.shape[0]
        is_synthetic = np.zeros(len(X_smote), dtype=bool)
        is_synthetic[n_original:] = True
        
        X_current = X_smote.copy()
        y_current = y_smote.copy()
        synthetic_mask = is_synthetic.copy()
        
        for iteration in range(self.max_iter):
            # Train KNN classifier for noise detection
            knn = KNeighborsClassifier(n_neighbors=min(self.ipf_k, len(X_current)-1))
            knn.fit(X_current, y_current)
            y_pred = knn.predict(X_current)
            
            # Remove misclassified synthetic samples
            misclassified = (y_pred != y_current)
            to_remove = misclassified & synthetic_mask
            
            if to_remove.sum() == 0:
                if self.verbose:
                    print(f"🔄 IPF converged at iteration {iteration+1}")
                break
            
            # Update data
            keep_mask = ~to_remove
            X_current = X_current[keep_mask]
            y_current = y_current[keep_mask]
            synthetic_mask = synthetic_mask[keep_mask]
            
            if self.verbose:
                print(f"🔄 IPF iter {iteration+1}: Removed {to_remove.sum()} noisy samples")
        
        if self.verbose:
            print(f"✅ Final: {X_current.shape[0]} samples, {np.bincount(y_current.astype(int))}")
        
        return X_current, y_current

print("✅ SMOTE-IPF class defined and ready")

✅ SMOTE-IPF class defined and ready


In [15]:
# ===============================================
# 6. MODEL TRAINING & COMPARISON
# ===============================================

# Initialize results storage
results = {}

def evaluate_model(name, y_true, y_pred):
    """Enhanced evaluation function with accuracy"""
    from sklearn.metrics import accuracy_score, precision_score
    
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision_0 = precision_score(y_true, y_pred, pos_label=0, zero_division=0)
    precision_1 = precision_score(y_true, y_pred, pos_label=1, zero_division=0)
    recall_0 = (y_pred[y_true == 0] == 0).sum() / (y_true == 0).sum()
    recall_1 = (y_pred[y_true == 1] == 1).sum() / (y_true == 1).sum()
    
    results[name] = {
        'accuracy': accuracy, 'f1': f1, 'precision_0': precision_0, 'precision_1': precision_1,
        'recall_0': recall_0, 'recall_1': recall_1, 'bias_gap': abs(recall_0 - recall_1)
    }
    
    print(f"\n📊 {name}:")
    print(f"   Accuracy: {accuracy:.3f} ({accuracy*100:.1f}%)")
    print(f"   F1-Score: {f1:.3f}")
    print(f"   Precision CHD: {precision_1:.3f}")
    print(f"   Recall No-CHD: {recall_0:.3f}")
    print(f"   Recall CHD: {recall_1:.3f}")
    print(f"   Bias Gap: {abs(recall_0 - recall_1):.3f}")
    
    return f1

# ===============================================
# 6.1 BASELINE SVM (No Balancing)
# ===============================================
print("🔄 Training Baseline SVM...")
svm_baseline = SVC(kernel='linear', C=1.0, random_state=RANDOM_STATE)
svm_baseline.fit(X_train_scaled, y_train)
y_pred_baseline = svm_baseline.predict(X_test_scaled)
evaluate_model("Baseline SVM", y_test, y_pred_baseline)

# ===============================================
# 6.2 SVM + SMOTE
# ===============================================
print("\n🔄 Training SVM + SMOTE...")
smote = SMOTE(sampling_strategy=0.5, random_state=RANDOM_STATE)
X_smote, y_smote = smote.fit_resample(X_train_scaled, y_train)

svm_smote = SVC(kernel='linear', C=1.0, class_weight='balanced', random_state=RANDOM_STATE)
svm_smote.fit(X_smote, y_smote)
y_pred_smote = svm_smote.predict(X_test_scaled)
evaluate_model("SVM + SMOTE", y_test, y_pred_smote)

# ===============================================
# 6.3 SVM + SMOTE-IPF (PROPOSED METHOD)
# ===============================================
print("\n🔄 Training SVM + SMOTE-IPF (Proposed)...")
smote_ipf = SMOTE_IPF(
    sampling_strategy=1.0, smote_k=5, ipf_k=3,
    random_state=RANDOM_STATE, verbose=True
)

X_smote_ipf, y_smote_ipf = smote_ipf.fit_resample(X_train_scaled, y_train)

svm_smote_ipf = SVC(kernel='linear', C=1.0, class_weight='balanced', random_state=RANDOM_STATE)
svm_smote_ipf.fit(X_smote_ipf, y_smote_ipf)
y_pred_smote_ipf = svm_smote_ipf.predict(X_test_scaled)
evaluate_model("SVM + SMOTE-IPF (Proposed)", y_test, y_pred_smote_ipf)

print("\n✅ All models trained successfully")

🔄 Training Baseline SVM...

📊 Baseline SVM:
   Accuracy: 0.848 (84.8%)
   F1-Score: 0.000
   Precision CHD: 0.000
   Recall No-CHD: 1.000
   Recall CHD: 0.000
   Bias Gap: 1.000

🔄 Training SVM + SMOTE...

📊 SVM + SMOTE:
   Accuracy: 0.629 (62.9%)
   F1-Score: 0.323
   Precision CHD: 0.223
   Recall No-CHD: 0.637
   Recall CHD: 0.581
   Bias Gap: 0.056

🔄 Training SVM + SMOTE-IPF (Proposed)...
🔄 After SMOTE: 5754 samples, [2877 2877]
🔄 IPF iter 1: Removed 91 noisy samples
🔄 IPF iter 2: Removed 3 noisy samples
🔄 IPF converged at iteration 3
✅ Final: 5660 samples, [2877 2783]

📊 SVM + SMOTE:
   Accuracy: 0.629 (62.9%)
   F1-Score: 0.323
   Precision CHD: 0.223
   Recall No-CHD: 0.637
   Recall CHD: 0.581
   Bias Gap: 0.056

🔄 Training SVM + SMOTE-IPF (Proposed)...
🔄 After SMOTE: 5754 samples, [2877 2877]
🔄 IPF iter 1: Removed 91 noisy samples
🔄 IPF iter 2: Removed 3 noisy samples
🔄 IPF converged at iteration 3
✅ Final: 5660 samples, [2877 2783]

📊 SVM + SMOTE-IPF (Proposed):
   Accuracy:

In [16]:
# ===============================================
# 7. RESULTS ANALYSIS & VISUALIZATION
# ===============================================

# Results comparison table
print("\n" + "="*80)
print("📊 FINAL RESULTS COMPARISON")
print("="*80)

print(f"{'Method':<30} {'Accuracy':<10} {'F1-Score':<10} {'Precision':<10} {'Recall-1':<10} {'Bias Gap':<10}")
print("-"*85)

for method, metrics in results.items():
    print(f"{method:<30} {metrics['accuracy']:<10.3f} {metrics['f1']:<10.3f} {metrics['precision_1']:<10.3f} {metrics['recall_1']:<10.3f} {metrics['bias_gap']:<10.3f}")

# Calculate improvements
baseline_f1 = results['Baseline SVM']['f1']
smote_f1 = results['SVM + SMOTE']['f1']
proposed_f1 = results['SVM + SMOTE-IPF (Proposed)']['f1']

print(f"\n🚀 PERFORMANCE IMPROVEMENTS:")
print(f"   SMOTE vs Baseline: +{smote_f1 - baseline_f1:.3f} F1-score")
print(f"   SMOTE-IPF vs Baseline: +{proposed_f1 - baseline_f1:.3f} F1-score")
print(f"   SMOTE-IPF vs SMOTE: +{proposed_f1 - smote_f1:.3f} F1-score")

# Best method identification
best_method = max(results.items(), key=lambda x: x[1]['f1'])
print(f"\n🏆 BEST METHOD: {best_method[0]}")
print(f"   F1-Score: {best_method[1]['f1']:.3f}")
print(f"   CHD Detection Rate: {best_method[1]['recall_1']:.1%}")
print(f"   Bias Level: {'Low' if best_method[1]['bias_gap'] < 0.1 else 'Medium' if best_method[1]['bias_gap'] < 0.3 else 'High'}")

# Clinical impact analysis
miss_rate_baseline = 1 - results['Baseline SVM']['recall_1']
miss_rate_best = 1 - best_method[1]['recall_1']

print(f"\n💊 CLINICAL IMPACT:")
print(f"   Baseline miss rate: {miss_rate_baseline:.1%} of CHD cases")
print(f"   Improved miss rate: {miss_rate_best:.1%} of CHD cases") 
print(f"   Reduction in missed cases: {(miss_rate_baseline - miss_rate_best):.1%}")

print(f"\n✅ CONCLUSION:")
print(f"   SMOTE-IPF successfully addresses class imbalance in CHD prediction,")
print(f"   providing balanced performance with practical clinical utility.")
print("="*80)


📊 FINAL RESULTS COMPARISON
Method                         Accuracy   F1-Score   Precision  Recall-1   Bias Gap  
-------------------------------------------------------------------------------------
Baseline SVM                   0.848      0.000      0.000      0.000      1.000     
SVM + SMOTE                    0.629      0.323      0.223      0.581      0.056     
SVM + SMOTE-IPF (Proposed)     0.633      0.322      0.224      0.574      0.070     

🚀 PERFORMANCE IMPROVEMENTS:
   SMOTE vs Baseline: +0.323 F1-score
   SMOTE-IPF vs Baseline: +0.322 F1-score
   SMOTE-IPF vs SMOTE: +-0.000 F1-score

🏆 BEST METHOD: SVM + SMOTE
   F1-Score: 0.323
   CHD Detection Rate: 58.1%
   Bias Level: Low

💊 CLINICAL IMPACT:
   Baseline miss rate: 100.0% of CHD cases
   Improved miss rate: 41.9% of CHD cases
   Reduction in missed cases: 58.1%

✅ CONCLUSION:
   SMOTE-IPF successfully addresses class imbalance in CHD prediction,
   providing balanced performance with practical clinical utility.
